In [1]:
import os
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from datetime import datetime
import dateutil.parser

import pandas as pd
import psycopg2 as pg
import pandas.io.sql as pd_sql

# Without a typical browser user agent senate.gov will reject the request
my_headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'}

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
url='https://www.senate.gov/general/contact_information/senators_cfm.cfm'
response = requests.get(url, headers=my_headers)
response.status_code

200

In [3]:
page = response.text
soup = BeautifulSoup(page, "lxml")

The ids in the list match the ids from the ipums dataset:

In [4]:
states_list = [
['Alabama', 'AL', 1], ['Alaska', 'AK', 2], ['Arizona', 'AZ', 4], ['Arkansas', 'AR', 5],
['California', 'CA', 6], ['Colorado', 'CO', 8], ['Connecticut', 'CT', 9], ['Delaware', 'DE', 10],
['District of Columbia', 'DC', 11], ['Florida', 'FL', 12], ['Georgia', 'GA', 13],
['Hawaii', 'HI', 15], ['Idaho', 'ID', 16], ['Illinois', 'IL', 17], ['Indiana', 'IN', 18],
['Iowa', 'IA', 19], ['Kansas', 'KS', 20], ['Kentucky', 'KY', 21], ['Louisiana', 'LA', 22],
['Maine', 'ME', 23], ['Maryland', 'MD', 24], ['Massachusetts', 'MA', 25], ['Michigan', 'MI', 26],
['Minnesota', 'MN', 27], ['Mississippi', 'MS', 28],
['Missouri', 'MO', 29], ['Montana', 'MT', 30], ['Nebraska', 'NE', 31], ['Nevada', 'NV', 32],
['New Hampshire', 'NH', 33], ['New Jersey', 'NJ', 34], ['New Mexico', 'NM', 35], 
['New York', 'NY', 36], ['North Carolina', 'NC', 37],
['North Dakota', 'ND', 38], ['Ohio', 'OH', 39], ['Oklahoma', 'OK', 40], ['Oregon', 'OR', 41],
['Pennsylvania', 'PA', 42], ['Rhode Island', 'RI', 44],
['South Carolina', 'SC', 45], ['South Dakota', 'SD', 46], ['Tennessee', 'TN', 47], ['Texas', 'TX', 48],
['Utah', 'UT', 49], ['Vermont', 'VT', 50], ['Virginia', 'VA', 51], ['Washington', 'WA', 53],
['West Virginia', 'WV', 54], ['Wisconsin', 'WI', 55], ['Wyoming', 'WY', 56]
]

In [5]:
import re
total_regex = re.compile(' - ')
senators = soup.find_all(text=total_regex)
senators_list = []
for dtg_string in senators:
    senname = dtg_string.findPreviousSibling().text.strip()
    m = re.match("([\w -]+), ([\w -]+)", senname)
    senfirstname = m.groups()[1]
    senlastname = m.groups()[0]

    senparty = dtg_string.strip()
    m = re.match("- \(([RDI]) - ([A-Z]{2})", senparty)
    senparty = m.groups()[0]
    senstate_abbr = m.groups()[1]
    
    for row in states_list:
        state = row[0]
        state_abbr = row[1]
        state_id = row[2]
        
        if state_abbr == senstate_abbr:
            break

    senators_list.append([senfirstname, senlastname, senparty, senstate_abbr, state, state_id])

In [2]:
POSTGRES_HOST = os.getenv('POSTGRES_HOST')
POSTGRES_PORT = os.getenv('POSTGRES_PORT')
POSTGRES_USER = os.getenv('POSTGRES_USER')
POSTGRES_HC_DB = os.getenv('POSTGRES_HC_DB')

connection_args = {
    'host': POSTGRES_HOST,
    'user': POSTGRES_USER,
    'dbname': POSTGRES_HC_DB,
    'port': POSTGRES_PORT
}

connection = pg.connect(**connection_args)

In [7]:
cursor = connection.cursor()

# Delete table if exists
query = "TRUNCATE senators RESTART IDENTITY;"
cursor.execute(query)
connection.commit()

for fname, lname, party, state_abbr, state, state_id in senators_list:
    query =  "INSERT INTO senators (fname, lname, party, state_abbr, state, statefip) VALUES (%s, %s, %s, %s, %s, %s);"
    data = (fname, lname, party, state_abbr, state, state_id)

    cursor.execute(query, data)
    connection.commit()

cursor.close()

In [8]:
query = "SELECT party, state_abbr, statefip, '' AS party_cnt FROM senators"
dfs = pd_sql.read_sql(query, connection)

In [9]:
# Add `party_cnt` column: 1=split ticket, 2=Democrat (both), 3=Republican (both)
dfs = dfs.groupby(['state_abbr', 'party']).agg({'statefip': 'first', 'party_cnt':'count'}).reset_index()
dfs.loc[(dfs['party'] == 'R') & (dfs['party_cnt'] == 2), 'party_cnt'] = 3
dfs = dfs.groupby(['statefip']).agg({'party_cnt': 'first'}).reset_index()

In [10]:
dfs.to_csv('data/sen_state_party_split.csv', index=None)